In [1]:
# Запустить контейнер в терминале
# docker run -p 5433:5433 jbfavre/vertica:latest

In [2]:
import time
import statistics
import uuid
from random import randint
import vertica_python

In [3]:
connection_info = {
    "host": "127.0.0.1",
    "port": 5433,
    "user": "dbadmin",
    "password": "",
    "database": "docker",
    "autocommit": True,
}

In [4]:
def generate_random_data(n_rows: int, iteration: int):
    data = []
    for _ in range(iteration*n_rows):
        timestamp_movie = randint(1, 100)
        time = randint(1000000000, 10000000000)
        data.append((str(uuid.uuid4()), str(uuid.uuid4()), timestamp_movie, time))

    yield data

In [5]:
class profiler():
    def __init__(self):
        self.start = time.time()
        self.all_durations = []
        
    def setup_start_time(self):
        self.start = time.time()

    def checkpoint(self):
        end_time = time.time()
        self.all_durations.append(end_time - self.start)
        self.start = end_time

    def __enter__(self):
        return self

    def __exit__(self, type, value, traceback):
        print(f"Median - {statistics.median(self.all_durations)}")
        avg = sum(self.all_durations) / len(self.all_durations)
        print(f"Average - {avg}")
        print(f"Summary - {sum(self.all_durations)}")

In [7]:
with vertica_python.connect(**connection_info) as connection:
    cursor = connection.cursor()
    
    cursor.execute(
        """CREATE TABLE IF NOT EXISTS views (
        id IDENTITY,
        user_id uuid NOT NULL,
        movie_id uuid NOT NULL,
        timestamp_movie integer NOT NULL,
        time integer NOT NULL);"""
    )

    print('\nТестирование вставки по 500 записей')
    with profiler() as p:
        for data in generate_random_data(500, 200):
            p.setup_start_time()
            cursor.executemany(
                "INSERT INTO views (user_id, movie_id, timestamp_movie, time) VALUES(?, ?, ?, ?)",
                data,
                use_prepared_statements=True,
            )
            p.checkpoint()

    print('\nТестирование одной вставки')
    with profiler() as p:
        for data in generate_random_data(1, 200):
            p.setup_start_time()
            cursor.executemany(
                "INSERT INTO views (user_id, movie_id, timestamp_movie, time) VALUES(?, ?, ?, ?)",
                data,
                use_prepared_statements=True,
            )
            p.checkpoint()

    print('\nТестирование получения данных')
    cursor.execute("SELECT user_id FROM views LIMIT 100;")
    uuids = [str(u[0]) for u in cursor.fetchall()]
    with profiler() as p:
        for u in uuids:
            p.setup_start_time()
            cursor.execute(f"SELECT * FROM views where user_id='{u}';")
            p.checkpoint()


Тестирование вставки по 500 записей
Median - 196.04894280433655
Average - 196.04894280433655
Summary - 196.04894280433655

Тестирование одной вставки
Median - 160.0019609928131
Average - 160.0019609928131
Summary - 160.0019609928131

Тестирование получения данных
Median - 0.006012916564941406
Average - 0.009008913040161133
Summary - 0.9008913040161133
